In [1]:
## Utiliza el API openmaps para identificar las cafeterias y 
## calcula el valor del viaje en UBER consultando el API de UBER

In [2]:
import requests
import json
import geopy as ge
import requests
import datetime
import pandas as pd
import folium
import numpy as np
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
from folium.features import DivIcon
import streamlit as st

In [3]:
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()


st.write(""" Titulo
""")

In [4]:
# query en el mapa
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query1 = """
[out:json];
//area["ISO3166-1"="PA"][admin_level=2];
area[name="Distrito Panamá"][admin_level=6];
(
node["amenity"="cafe"](area);
way["amenity"="cafe"](area);
rel["amenity"="cafe"](area);
);
out center;
"""

In [5]:
response1 = requests.get(overpass_url, params={'data': overpass_query1})

In [6]:
data = response1.json()

In [7]:
data

{'version': 0.6,
 'generator': 'Overpass API 0.7.59 e21c39fe',
 'osm3s': {'timestamp_osm_base': '2023-01-17T13:35:39Z',
  'timestamp_areas_base': '2023-01-17T12:23:32Z',
  'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'},
 'elements': [{'type': 'node',
   'id': 858116224,
   'lat': 9.0094142,
   'lon': -79.5364683,
   'tags': {'amenity': 'cafe', 'name': 'Nikos'}},
  {'type': 'node',
   'id': 860727292,
   'lat': 8.9692564,
   'lon': -79.5609382,
   'tags': {'amenity': 'cafe', 'name': 'Kotowa Coffee Store'}},
  {'type': 'node',
   'id': 874794299,
   'lat': 8.9892264,
   'lon': -79.5317927,
   'tags': {'amenity': 'cafe', 'name': 'Cafe Duran'}},
  {'type': 'node',
   'id': 879925244,
   'lat': 8.9154675,
   'lon': -79.5266176,
   'tags': {'amenity': 'cafe',
    'cuisine': 'ice_cream',
    'name': 'Gelarti - Helados Gourmet',
    'source': 'KG Ground Survey 2015'}},
  {'type': 'node',
   'id': 1538866373,
   'lat': 8.

In [8]:
service = ge.Nominatim(user_agent = "paracredicorp@gmail.com")
origen = service.geocode("Via Argentina, Panama")
#print(origen.latitude)
#print(origen.longitude)

In [9]:
data = data.copy()
coords = []
for element in data['elements']:
 if element['type'] == 'node':
        lon = element['lon']
        lat = element['lat']
        coords.append((lon, lat))
 elif 'center' in element:
        lon = element['center']['lon']
        lat = element['center']['lat']
        coords.append((lon, lat))
# Convert coordinates into numpy array
X = np.array(coords)

In [10]:
result = pd.DataFrame()

In [11]:
def get_ride_price(origin_latitude, origin_longitude, destination_latitude, destination_longitude):
      
    url = "https://www.uber.com/api/loadFEEstimates?localeCode=en"

    payload = json.dumps({
      "origin": {
        "latitude": origin_latitude,
        "longitude": origin_longitude
      },
      "destination": {
        "latitude": destination_latitude,
        "longitude": destination_longitude
      },
      "locale": "en"
    })
    headers = {
      'content-type': 'application/json',
      'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36',
      'x-csrf-token': 'x'
    }

    response = requests.request("POST", url, headers=headers, data=payload)
      
    result1 = [[ datetime.datetime.now(), destination_latitude, destination_longitude,
               x['vehicleViewDisplayName'], x['fareString']] for x in response.json()['data']['prices']]
    result1 = pd.DataFrame(result1)
    result1.rename(columns= { 0: 'fecha', 1:'latitud', 2:'longitud',3: 'tipo', 4:'valor' }, inplace=True)
    global result
    result = result.append(result1, ignore_index=True)

In [12]:
#get_ride_price(origen.latitude, origen.longitude, X[0,1], X[0,0])
#print(result)

In [13]:
for i in range(25):
#for i in range(len(X)):
    get_ride_price(origen.latitude, origen.longitude, X[i,1], X[i,0])

In [14]:
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF8')
result['valor1']= result['valor'].apply(lambda x: locale.atof(x.strip('$')))

In [15]:
result['lugar2'] = result['latitud'].apply(str) + "," + result['longitud'].apply(str)

2023-01-17 08:37:54.479 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


In [16]:
destinos2 = pd.DataFrame(result['lugar2'].unique())
destinos2.columns = ['lugar2']
#destinos2.head()

In [17]:
geolocator = ge.Nominatim(user_agent = "paracredicorp@gmail.com")
#lugar2 = geolocator.reverse("8.980706,-79.527544")
#print(lugar2.address)

In [18]:
destinos2['destino']= destinos2['lugar2'].apply(lambda x: geolocator.reverse(x))
#destinos2

In [19]:
result2 = result.merge(destinos2, left_on='lugar2', right_on='lugar2')
#result2.shape

In [20]:
#result2.head()

In [21]:
result3 = pd.DataFrame()
for i in result2['tipo'].unique():
    result3 = result3.append(result2[result2['tipo']== i].sort_values(by=['valor1']).head(25))

In [22]:
#result3.shape

In [23]:
#result3.head()

In [24]:
for i in result3['destino']:
    result3['destino1'] = result3['destino'].astype('string').str.split(",").str[0]

In [25]:
result3['destino2'] = result3['destino1']+ "," + result['valor']

In [26]:
#result3.head()

In [27]:
m_3 = folium.Map(location=[origen.latitude,origen.longitude], tiles='openstreetmap', zoom_start=12)

folium.CircleMarker(
    location=[origen.latitude,origen.longitude],
    radius=10,
    popup=origen.address,
    color="crimson",
    fill=True,
    fill_color=False,
).add_to(m_3)

# Añadir puntos en el mapa
for idx, row in result3.iterrows():
    Marker([row['latitud'], row['longitud']], row['destino2']).add_to(m_3)
    
m_3